In [71]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l1
from tensorflow.keras.layers import Dropout

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

In [72]:
# Cargar el dataset Iris
iris = load_iris()
X = iris.data  # Características (150 muestras, 4 características)
y = iris.target  # Etiquetas (especies)

# Convertir y a una variable continua (por ejemplo, usar el ancho del pétalo como objetivo)
y = to_categorical(y)  # One-hot encoding (ej: [1, 0, 0] para setosa)

In [73]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_cv, X_test, y_cv, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [74]:
print(X_train.shape)
print(X_test.shape)
print(X_cv.shape)

(90, 4)
(30, 4)
(30, 4)


In [75]:
# Diseñar la red neuronal
model = Sequential([
    Dense(100, activation='relu', kernel_regularizer=l1(0.0001)),  # Regularización L1 - lambda 0.01
    Dense(100, activation='relu'),
    Dense(3, activation='softmax')  # Capa de salida
])

In [76]:
# Compilar el modelo
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

In [77]:
# Entrenar el modelo
#history =model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_cv, y_cv))

Epoch 1/20


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 223ms/step - accuracy: 0.3519 - loss: 0.2684 - val_accuracy: 0.4000 - val_loss: 0.2063
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.3852 - loss: 0.2084 - val_accuracy: 0.8000 - val_loss: 0.1702
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.6582 - loss: 0.1963 - val_accuracy: 0.8000 - val_loss: 0.1461
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.6442 - loss: 0.1861 - val_accuracy: 0.8000 - val_loss: 0.1347
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.6207 - loss: 0.1748 - val_accuracy: 0.8000 - val_loss: 0.1231
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.6481 - loss: 0.1490 - val_accuracy: 0.8000 - val_loss: 0.1165
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.6631 - loss: 0.1413 - val_accuracy: 0.8333 - val_loss: 0.1103
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.7215 - loss: 0.1309 - val_accuracy: 0.9333 - val_loss: 0.1025
Epoch 9/

In [78]:
# Evaluar el modelo en el conjunto de prueba
J_train = model.evaluate(X_train, y_train, verbose=0)
J_cv = model.evaluate(X_cv, y_cv, verbose=0)
J_test = model.evaluate(X_test, y_test, verbose=0)
print("J_train en el conjunto de prueba: ",J_train)
print("J_cv en el conjunto de prueba: ", J_cv)
print("J_test en el conjunto de prueba: ",J_test)

J_train en el conjunto de prueba:  [0.06481241434812546, 0.9555555582046509]
J_cv en el conjunto de prueba:  [0.04950566962361336, 1.0]
J_test en el conjunto de prueba:  [0.05930105596780777, 0.9666666388511658]


| Caso  | Interpretación | Solución        |
|---------|------|--------------|
| Jtrain≈Jtest≈Jcv​ y todos son altos     | Subajuste (underfitting)   | Aumentar la capacidad del modelo (más capas/neuronas)       |
| Jtrain​ bajo, pero Jtest≫Jtrain​	     | Sobreajuste (overfitting)   | Regularización, dropout o más datos |
|  Jtest≈Jcv​, pero ambos son un poco más altos que Jtrain  | ✅	Modelo bien generalizado    | No necesita cambios       |


In [79]:
# MSE de predecir siempre la media
y_mean = np.mean(y_train)
mse_baseline = np.mean((y_train - y_mean) ** 2)

print("MSE del modelo base:", mse_baseline)

MSE del modelo base: 0.22222222222222227


Si Jtrain ≫ MSEbase, el modelo no está aprendiendo bien.

Si Jtrain=0.24, pregúntate:

    ¿Cuál es la escala de las etiquetas reales?
        Si las salidas están en el rango [0,1], MSE = 0.24 es alto.
        Si las salidas están entre [0,1000], MSE = 0.24 es muy bajo

In [80]:
from tensorflow.keras.regularizers import l1

